# 💻 Step 2: Script Mode での機械学習

前のノートブックで作成・管理したS3データを使って、**Script Mode**での機械学習を実行します。

## 🎯 このノートブックで学ぶこと
- S3データの読み込みと活用
- ノートブック内での機械学習パイプライン
- 複数モデルの比較と評価
- Script Modeの特徴と利点

## ⏱️ 実行時間: 約5-8分

## 1. 環境設定とS3データの読み込み

In [ ]:
import sagemaker
import boto3
import pandas as pd
import numpy as np
import time
import json
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# 機械学習ライブラリ
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
import warnings
warnings.filterwarnings('ignore')

print("💻 Script Mode環境の設定")
print(f"📅 開始時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 前のノートブックで保存したS3パス情報を読み込み
try:
    with open('s3_data_paths.json', 'r') as f:
        s3_paths = json.load(f)
    print("✅ S3データパス情報を読み込み完了")
    for data_type, path in s3_paths.items():
        print(f"  📊 {data_type}: {path}")
except FileNotFoundError:
    print("❌ S3パス情報が見つかりません")
    print("🔄 先に 01_data_management.ipynb を実行してください")
    raise

## 2. S3からのデータ読み込み

### 📚 学習ポイント
- S3から直接DataFrameに読み込み
- データの基本確認
- 読み込み時間の測定

In [ ]:
print("📥 S3からデータを読み込み中...")
data_load_start = time.time()

# S3から各データセットを読み込み
datasets = {}
for data_type, s3_path in s3_paths.items():
    print(f"  📊 {data_type} データを読み込み中...")
    
    file_start = time.time()
    datasets[data_type] = pd.read_csv(s3_path)
    file_time = time.time() - file_start
    
    print(f"    ✅ 完了 ({file_time:.2f}秒, 形状: {datasets[data_type].shape})")

data_load_time = time.time() - data_load_start

print(f"\n📊 データ読み込み完了:")
print(f"  ⏱️ 総読み込み時間: {data_load_time:.2f}秒")
print(f"  📈 訓練データ: {datasets['train'].shape}")
print(f"  📉 検証データ: {datasets['validation'].shape}")
print(f"  🎯 テストデータ: {datasets['test'].shape}")

# データの基本確認
train_data = datasets['train']
val_data = datasets['validation']
test_data = datasets['test']

print(f"\n🔍 データ品質確認:")
print(f"  📊 クラス分布（訓練）: {dict(train_data['target'].value_counts().sort_index())}")
print(f"  🔍 欠損値（訓練）: {train_data.isnull().sum().sum()}個")
print(f"  📏 特徴量数: {len([col for col in train_data.columns if col != 'target'])}個")

## 3. データの可視化と理解

In [ ]:
# データの可視化
plt.figure(figsize=(15, 5))

# クラス分布
plt.subplot(1, 3, 1)
train_data['target'].value_counts().plot(kind='bar', color=['skyblue', 'lightgreen', 'salmon'])
plt.title('クラス分布（不均衡データ）')
plt.xlabel('クラス')
plt.ylabel('サンプル数')
plt.xticks(rotation=0)

# 重要特徴量の分布
plt.subplot(1, 3, 2)
for target in train_data['target'].unique():
    subset = train_data[train_data['target'] == target]
    plt.hist(subset['feature_00'].dropna(), alpha=0.7, label=f'Class {target}', bins=20)
plt.title('重要特徴量の分布')
plt.xlabel('feature_00')
plt.ylabel('頻度')
plt.legend()

# 欠損値パターン
plt.subplot(1, 3, 3)
missing_data = train_data.isnull().sum()
missing_features = missing_data[missing_data > 0]
if len(missing_features) > 0:
    missing_features.plot(kind='bar')
    plt.title('欠損値パターン')
    plt.xlabel('特徴量')
    plt.ylabel('欠損値数')
    plt.xticks(rotation=45)
else:
    plt.text(0.5, 0.5, '欠損値なし', ha='center', va='center', transform=plt.gca().transAxes)
    plt.title('欠損値パターン')

plt.tight_layout()
plt.show()

print("📊 データ理解完了: S3から読み込んだデータの特徴を確認しました")

## 4. Script Mode での機械学習実行

### 📚 学習ポイント
- 前処理パイプラインの構築
- 複数モデルの比較
- ハイパーパラメータ最適化
- 結果の評価と可視化

In [ ]:
def create_lecture_preprocessing_pipeline():
    """講義用：軽量前処理パイプライン"""
    return Pipeline([
        ('imputer', SimpleImputer(strategy='median')),    # 欠損値補完
        ('scaler', StandardScaler()),                     # 標準化
        ('feature_selection', SelectKBest(f_classif, k=20))  # 特徴選択
    ])

def train_lecture_models(X_train, y_train, X_val, y_val):
    """講義用：軽量モデル比較"""
    print("=== 💻 Script Mode: 機械学習実行開始 ===")
    print("⏱️ 予想実行時間: 3-6分")
    
    # パラメータ数を削減（講義用）
    models = {
        'RandomForest': {
            'model': RandomForestClassifier(random_state=42),
            'params': {
                'n_estimators': [50, 100],
                'max_depth': [10, 20]
            }
        },
        'GradientBoosting': {
            'model': GradientBoostingClassifier(random_state=42),
            'params': {
                'n_estimators': [50, 100],
                'learning_rate': [0.1, 0.2]
            }
        },
        'LogisticRegression': {
            'model': LogisticRegression(random_state=42, max_iter=500),
            'params': {
                'C': [0.1, 1.0]
            }
        }
    }
    
    results = {}
    best_models = {}
    
    for name, config in models.items():
        print(f"\n📊 {name} を最適化中...")
        start_time = time.time()
        
        # 軽量グリッドサーチ
        grid_search = GridSearchCV(
            config['model'],
            config['params'],
            cv=3,
            scoring='accuracy',
            n_jobs=-1,
            verbose=0
        )
        
        grid_search.fit(X_train, y_train)
        training_time = time.time() - start_time
        
        # 評価
        best_model = grid_search.best_estimator_
        train_score = best_model.score(X_train, y_train)
        val_score = best_model.score(X_val, y_val)
        
        # 軽量クロスバリデーション
        cv_scores = cross_val_score(best_model, X_train, y_train, cv=3)
        
        best_models[name] = best_model
        results[name] = {
            'best_params': grid_search.best_params_,
            'train_score': train_score,
            'val_score': val_score,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'training_time': training_time
        }
        
        print(f"  ✅ 最適パラメータ: {grid_search.best_params_}")
        print(f"  📈 検証精度: {val_score:.4f}")
        print(f"  ⏱️ 訓練時間: {training_time:.1f}秒")
    
    # 最適モデル選択
    best_model_name = max(results.keys(), key=lambda k: results[k]['val_score'])
    best_model = best_models[best_model_name]
    
    print(f"\n🏆 最適モデル: {best_model_name}")
    print(f"🎯 検証精度: {results[best_model_name]['val_score']:.4f}")
    
    return best_model, best_model_name, results

# Script Mode実行
print("🚀 Script Mode機械学習開始...")
script_start_time = time.time()

# データ準備
X_train = train_data.drop('target', axis=1)
y_train = train_data['target']
X_val = val_data.drop('target', axis=1)
y_val = val_data['target']
X_test = test_data.drop('target', axis=1)
y_test = test_data['target']

print(f"📊 データ形状確認:")
print(f"  📈 訓練: {X_train.shape}")
print(f"  📉 検証: {X_val.shape}")
print(f"  🎯 テスト: {X_test.shape}")

# 前処理
print(f"\n🔧 前処理実行中...")
preprocessing_start = time.time()
preprocessor = create_lecture_preprocessing_pipeline()
X_train_processed = preprocessor.fit_transform(X_train, y_train)
X_val_processed = preprocessor.transform(X_val)
X_test_processed = preprocessor.transform(X_test)
preprocessing_time = time.time() - preprocessing_start

print(f"✅ 前処理完了: {preprocessing_time:.1f}秒")
print(f"📉 特徴量数: {X_train.shape[1]} → {X_train_processed.shape[1]}")

# モデル訓練・比較
script_model, script_best_name, script_results = train_lecture_models(
    X_train_processed, y_train, X_val_processed, y_val
)

script_total_time = time.time() - script_start_time
print(f"\n🎉 Script Mode完了: {script_total_time:.1f}秒")

## 5. Script Mode結果の評価と可視化

In [ ]:
# Script Modeモデルの評価
print("=== 📊 Script Mode: 結果評価 ===")

script_predictions = script_model.predict(X_test_processed)
script_accuracy = accuracy_score(y_test, script_predictions)

print(f"🎯 テスト精度: {script_accuracy:.4f}")
print(f"\n📋 分類レポート:")
print(classification_report(y_test, script_predictions))

# 結果の可視化
plt.figure(figsize=(15, 5))

# 混同行列
plt.subplot(1, 3, 1)
cm = confusion_matrix(y_test, script_predictions)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title(f'混同行列 - {script_best_name}')
plt.xlabel('予測')
plt.ylabel('実際')

# モデル比較（精度）
plt.subplot(1, 3, 2)
model_names = list(script_results.keys())
val_scores = [script_results[name]['val_score'] for name in script_results]
colors = ['skyblue', 'lightgreen', 'salmon']
bars = plt.bar(model_names, val_scores, color=colors)
plt.title('モデル別検証精度')
plt.ylabel('精度')
plt.ylim(0, 1)
# 値をバーの上に表示
for bar, score in zip(bars, val_scores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{score:.3f}', ha='center', va='bottom')

# 訓練時間比較
plt.subplot(1, 3, 3)
training_times = [script_results[name]['training_time'] for name in script_results]
bars = plt.bar(model_names, training_times, color=colors)
plt.title('モデル別訓練時間')
plt.ylabel('時間（秒）')
# 値をバーの上に表示
for bar, time_val in zip(bars, training_times):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
             f'{time_val:.1f}s', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print(f"\n📈 Script Mode結果サマリー:")
print(f"  🏆 最適モデル: {script_best_name}")
print(f"  🎯 テスト精度: {script_accuracy:.4f}")
print(f"  ⏱️ 総実行時間: {script_total_time:.1f}秒")
print(f"  🔧 前処理時間: {preprocessing_time:.1f}秒")
print(f"  🤖 モデル訓練時間: {sum(training_times):.1f}秒")

# Script Mode情報を保存（比較用）
script_mode_info = {
    'best_model': script_best_name,
    'test_accuracy': script_accuracy,
    'total_time': script_total_time,
    'preprocessing_time': preprocessing_time,
    'training_times': training_times,
    'model_results': script_results
}

with open('script_mode_info.json', 'w') as f:
    json.dump(script_mode_info, f, indent=2, default=str)
    
print(f"\n💾 Script Mode情報を保存: script_mode_info.json")
print(f"🚀 次のノートブック: 03_training_jobs.ipynb")
print(f"   同じS3データを使ってTraining Jobsを実行します！")